In [23]:
#import required libraries
import pandas as pd
import string
from nltk.corpus import stopwords

In [24]:
#Get the spam data collection using pandas
#df_spamCollection = pd.read_csv("C:\\Users\\Harsh\\Documents\\SpamCollection",sep="\t")
df_spam_collection  = pd.read_csv("C:\\Users\\91808\\Downloads\\NLP(Spam Collection)",sep='\t',names=['response','message'])

In [25]:
#view first five records 
df_spam_collection.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [26]:
#view more information about the spam data using describe method
df_spam_collection.describe()

,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [27]:
#view response using group by and describe method
df_spam_collection.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [28]:
#verify length of the message and also add it also as a new column (feature)
df_spam_collection['length'] = df_spam_collection['message'].apply(len)

In [29]:
#view first 5 message with length
df_spam_collection.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [30]:
#define a function to get rid of stopwords present in the message 
def message_text_process(mess):
    #check characters to see if there are punctuations
    no_punctuation = [char for char in mess if char not in string.punctuation]
    #now form the sentance 
    no_punctuation = ''.join(no_punctuation)
    #Now eliminate any stopwords
    return [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]

In [31]:
#verify that function is working
df_spam_collection['message'].head(5).apply(message_text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [32]:
#start text processing with vectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [33]:
#bag of words by applying the function and fit the data (message) into it 
bag_of_words_transformer = CountVectorizer(analyzer=message_text_process).fit(df_spam_collection['message'])

In [34]:
#print the length of bag of words stored in the vocabulary_attribute
print(len(bag_of_words_transformer.vocabulary_))

11425


In [35]:
#store bag of words for message using transform method
message_bagofwords = bag_of_words_transformer.transform(df_spam_collection['message'])

In [36]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(message_bagofwords)

In [37]:
#print the shape of the tfidf 
message_tfidf = tfidf_transformer.transform(message_bagofwords)
print(message_tfidf.shape)

(5572, 11425)


In [38]:
#choose naive bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spam_detection_model = MultinomialNB().fit(message_tfidf,df_spam_collection['response'])

In [39]:
#check the model for the predicted and expected value say for message#2 and message#5['message'][2]
message = df_spam_collection['message'][2]
bag_of_words_for_message = bag_of_words_transformer.transform([message])
tfidf = tfidf_transformer.transform(bag_of_words_for_message)

print('predicted',spam_detection_model.predict(tfidf)[0])
print('expected',df_spam_collection.response[2])

predicted spam
expected spam
